In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 100)

In [23]:
DATA_PATH = "Data/processed/311_legacy_raw_merged.csv"

df = pd.read_csv(DATA_PATH, low_memory=False)

print("Rows:", df.shape[0])
print("Columns:", df.shape[1])
df.head()

Rows: 3284507
Columns: 30


,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,closure_reason,case_title,subject,reason,type,queue,department,submitted_photo,closed_photo,location,fire_district,pwd_district,city_council_district,police_district,neighborhood,neighborhood_services_district,ward,precinct,location_street_name,location_zipcode,latitude,longitude,geom_4326,source
0,101003148265,2020-01-01 00:07:00,2020-01-13 03:30:00,2020-01-20 06:43:10,OVERDUE,Closed,Case Closed. Closed date : 2020-01-20 11:43:10...,Park Lights - Doherty Playground (BPRD),Parks & Recreation Department,Park Maintenance & Safety,Parks Lighting/Electrical Issues,INFO_Reallocation From Dept,INFO,NaN,NaN,INTERSECTION of S Quincy Pl & Saint Martin St ...,3,1A,1,A15,Charlestown,2,2,0207,INTERSECTION S Quincy Pl & Saint Martin St,NaN,42.382987,-71.067828,0101000020E6100000ADA5774957C451C01A77A6B30531...,Citizens Connect App
1,101003148266,2020-01-01 00:54:48,2020-01-01 00:54:48,2020-01-02 01:11:49,OVERDUE,Closed,Case Closed. Closed date : 2020-01-02 06:11:49...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,Parking Enforcement,BTDT_Parking Enforcement,BTDT,NaN,NaN,85 Bloomfield St Dorchester MA 02124,7,03,4,C11,Dorchester,8,Ward 17,1702,85 Bloomfield St,2124.0,42.298900,-71.069541,0101000020E6100000E9E56F5A73C451C07E7212574226...,Citizens Connect App
2,101003148268,2020-01-01 01:14:13,2020-01-03 03:30:00,2020-01-01 10:05:46,ONTIME,Closed,Case Closed. Closed date : 2020-01-01 15:05:46...,Requests for Street Cleaning,Public Works Department,Street Cleaning,Requests for Street Cleaning,PWDx_District 1C: Downtown,PWDx,NaN,NaN,423 Shawmut Ave Roxbury MA 02118,4,1C,2,D4,South End,6,Ward 9,0901,423 Shawmut Ave,2118.0,42.340130,-71.074291,0101000020E6100000E2300A2DC1C451C0AAFE665D892B...,Citizens Connect App
3,101003148269,2020-01-01 01:19:00,2020-01-03 03:30:00,2020-01-02 01:10:56,ONTIME,Closed,Case Closed. Closed date : 2020-01-02 06:10:56...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,Parking Enforcement,BTDT_Parking Enforcement,BTDT,NaN,NaN,6 Moreland St Roxbury MA 02119,7,10B,7,B2,Roxbury,13,Ward 12,1201,6 Moreland St,2119.0,42.325960,-71.082761,0101000020E61000006A0602F34BC551C0EB78EE0AB929...,Constituent Call
4,101003148271,2020-01-01 02:02:00,2020-01-03 03:30:00,2020-01-01 02:07:17,ONTIME,Closed,Case Closed. Closed date : 2020-01-01 07:07:17...,Missed Trash: District 1B,Public Works Department,Sanitation,Missed Trash/Recycling/Yard Waste/Bulk Item,PWDx_Missed Trash\Recycling\Yard Waste\Bulk Item,PWDx,NaN,NaN,25-29 Charles St Boston MA 02114,3,1B,8,A1,Beacon Hill,14,Ward 5,0503,25-29 Charles St,2114.0,42.356920,-71.069650,0101000020E61000001AD6442775C451C0A731108AAF2D...,Constituent Call


In [24]:
sorted(df.columns.tolist())

['case_enquiry_id',
 'case_status',
 'case_title',
 'city_council_district',
 'closed_dt',
 'closed_photo',
 'closure_reason',
 'department',
 'fire_district',
 'geom_4326',
 'latitude',
 'location',
 'location_street_name',
 'location_zipcode',
 'longitude',
 'neighborhood',
 'neighborhood_services_district',
 'on_time',
 'open_dt',
 'police_district',
 'precinct',
 'pwd_district',
 'queue',
 'reason',
 'sla_target_dt',
 'source',
 'subject',
 'submitted_photo',
 'type',
 'ward']

## Missing Values

In [25]:
critical_cols = [
'open_dt',
'closed_dt',
'sla_target_dt',
'case_status',
'on_time',
'department',
'type',
'neighborhood'
]

missing_pct = df[critical_cols].isna().mean().sort_values(ascending=False)
missing_pct

sla_target_dt    0.142118
closed_dt        0.092117
neighborhood     0.000944
on_time          0.000204
open_dt          0.000000
case_status      0.000000
department       0.000000
type             0.000000
dtype: float64

## Analyze Data Quality 

### Closed Date earlier than Open Date

In [45]:
df_dates = df.copy()

df_dates['open_dt_parsed'] = pd.to_datetime(df_dates['open_dt'], errors='coerce')
df_dates['closed_dt_parsed'] = pd.to_datetime(df_dates['closed_dt'], errors='coerce')

invalid_dates = df_dates[
df_dates['open_dt_parsed'].notna() &
df_dates['closed_dt_parsed'].notna() &
(df_dates['closed_dt_parsed'] < df_dates['open_dt_parsed'])
]

invalid_dates[['case_enquiry_id', 'open_dt', 'closed_dt']].head(10)


col_with_dates = (df_dates['open_dt_parsed'].notna() &
df_dates['closed_dt_parsed'].notna())

valid_count = len(col_with_dates) - len(invalid_dates)
invalid_count = len(invalid_dates)


print("Percentage of invalid date records: {:.4f}%".format(invalid_count / len(df_dates) * 100))


Percentage of invalid date records: 0.0241%


## Rows eligible for SLA analysis.


In [46]:
sla_df = df.loc[
    (df['case_status'] == 'Closed') &
    df['sla_target_dt'].notna()
].copy()

print("Total closed cases with SLA target date:", sla_df.shape[0])
print("Percentage of total records:", sla_df.shape[0] / len(df) * 100)

Total closed cases with SLA target date: 2563874
Percentage of total records: 78.0596296491376


## Assess whether the SLA outcome field (on_time) is reliable.


In [42]:
mask = (
    (df['case_status'] == 'Closed') &
    df['sla_target_dt'].notna() &
    df['closed_dt'].notna() &
    df['on_time'].isna()
)

mask.mean()


0.00020368353606797

#### Less than 0.1% of closed cases with defined SLA targets lack an on_time value, indicating minor inconsistencies in the source system’s derived fields. To ensure correctness and consistency across years, SLA compliance was recomputed from raw timestamps

In [12]:
df['computed_on_time'] = np.where(
    (df['case_status'] == 'Closed') &
    df['sla_target_dt'].notna() &
    df['closed_dt'].notna(),
    np.where(
        df['closed_dt'] <= df['sla_target_dt'],
        'ONTIME',
        'OVERDUE'
    ),
    np.nan
)
df[['case_enquiry_id', 'case_status', 'closed_dt', 'sla_target_dt', 'on_time', 'computed_on_time']].head(10)

,case_enquiry_id,case_status,closed_dt,sla_target_dt,on_time,computed_on_time
0,101003148265,Closed,2020-01-20 06:43:10,2020-01-13 03:30:00,OVERDUE,OVERDUE
1,101003148266,Closed,2020-01-02 01:11:49,2020-01-01 00:54:48,OVERDUE,OVERDUE
2,101003148268,Closed,2020-01-01 10:05:46,2020-01-03 03:30:00,ONTIME,ONTIME
3,101003148269,Closed,2020-01-02 01:10:56,2020-01-03 03:30:00,ONTIME,ONTIME
4,101003148271,Closed,2020-01-01 02:07:17,2020-01-03 03:30:00,ONTIME,ONTIME
5,101003148272,Closed,2020-01-01 03:11:52,2020-01-03 03:30:00,ONTIME,ONTIME
6,101003148273,Closed,2020-01-02 01:09:58,2020-01-03 03:30:00,ONTIME,ONTIME
7,101003148274,Closed,2020-01-01 03:19:48,NaN,ONTIME,nan
8,101003148276,Closed,2020-01-02 00:08:04,2020-01-03 03:30:00,ONTIME,ONTIME
9,101003148277,Closed,2020-01-01 03:48:39,2020-01-06 03:30:00,ONTIME,ONTIME


## on_time exists but SLA missing


In [43]:
contradictions = df[
    (df['on_time'].notna()) & 
    (df['sla_target_dt'].isna())
]

len(contradictions), len(contradictions) / len(df)

print("Total contradictory records:", len(contradictions))
print("Percentage of contradictory records: {:.4f}%".format(len(contradictions) / len(df) * 100))

Total contradictory records: 466789
Percentage of contradictory records: 14.2118%
